## Diagnóstico

Raúl Diaz Campos | 1720688J | Sección 3

In [87]:
import json
import re

In [2]:
FILE = "./farmers-protest-tweets-2021-03-5.json"

In [41]:
def get_tweets(FILE):
    return [json.loads(tweet) for tweet in open(FILE)]

In [6]:
tweets = get_tweets(FILE)

In [7]:
# Primera función
def retweeted_tweets(tweets):
    def retweet(tweet):
        return tweet["retweetCount"]
    
    most_retweeted = []
    
    for tweet in tweets:
            if len(most_retweeted) <= 9:
                most_retweeted.append(tweet)
                most_retweeted = sorted(most_retweeted, key=retweet)
            else:
                if most_retweeted[9]["retweetCount"] <= tweet["retweetCount"]:
                    most_retweeted.pop(0)
                    most_retweeted.append(tweet)
                elif most_retweeted[0]["retweetCount"] <= tweet["retweetCount"]:
                    most_retweeted.append(tweet)
                    most_retweeted = sorted(most_retweeted, key=retweet)[1:]
    return most_retweeted

In [44]:
[{"tweet": tweet["id"], "retweets": tweet["retweetCount"]} for tweet in retweeted_tweets(tweets)][::-1]

[{'tweet': 1356625889602199552, 'retweets': 315547},
 {'tweet': 1356694884615340037, 'retweets': 103957},
 {'tweet': 1357282507616645122, 'retweets': 67694},
 {'tweet': 1356848397899112448, 'retweets': 35921},
 {'tweet': 1356827705161879553, 'retweets': 26972},
 {'tweet': 1357048037302960129, 'retweets': 23251},
 {'tweet': 1357755699162398720, 'retweets': 20132},
 {'tweet': 1356681136655769605, 'retweets': 18744},
 {'tweet': 1357009721090138112, 'retweets': 17368},
 {'tweet': 1359013362881994752, 'retweets': 15677}]

In [74]:
# Segunda función
def users_with_more_tweets(tweets, statuses=False): # statuses=True indica que debemos obtener los tweets desde los datos
    def more_tweets(user):
        return user["statusesCount"]
    
    if statuses:
    
        users = {}
        with_more_tweets = []

        for tweet in tweets:
            if not tweet["user"]["id"] in users:
                users[tweet["user"]["id"]] = tweet["user"]
                if len(with_more_tweets) <= 9:
                      with_more_tweets.append(tweet["user"])
                      with_more_tweets = sorted(with_more_tweets, key=more_tweets)
                else: 
                    if with_more_tweets[9]["statusesCount"] <= tweet["user"]["statusesCount"]:
                        with_more_tweets.pop(0)
                        with_more_tweets.append(tweet["user"])
                    elif with_more_tweets[0]["statusesCount"] <= tweet["user"]["statusesCount"]:
                        with_more_tweets.append(tweet["user"])
                        with_more_tweets = sorted(with_more_tweets, key=more_tweets)[1:]
        return with_more_tweets
    
    else:
        
        users = {}
        with_more_tweets = []
        
        for tweet in tweets:
            if tweet["user"]["id"] in users:
                users[tweet["user"]["id"]]["tweets"] += 1
                
            else:
                users[tweet["user"]["id"]] = {k: tweet["user"][k] for k in tweet["user"]}
                users[tweet["user"]["id"]]["tweets"] = 1
                                
        return sorted([users[u] for u in users], key=lambda i: i["tweets"])[-1:-10:-1]
    
    


In [75]:
[{"id": user["id"], "name": user["username"], "tweets": user["tweets"]} for user in users_with_more_tweets(tweets)]

[{'id': 980633168981061632, 'name': 'harjot_tweeting', 'tweets': 7134},
 {'id': 69027875, 'name': 'tasveersandhu', 'tweets': 2091},
 {'id': 2930682630, 'name': 'shells_n_petals', 'tweets': 1991},
 {'id': 452391771, 'name': 'jot__b', 'tweets': 1841},
 {'id': 1354532795847073796, 'name': 'rebelpacifist', 'tweets': 1806},
 {'id': 48085084, 'name': 'rumsomal', 'tweets': 1722},
 {'id': 872085660886282240, 'name': 'Iamjazzie96', 'tweets': 1502},
 {'id': 1340186972615307264, 'name': 'Jass_k_G', 'tweets': 1460},
 {'id': 1309302936317636608, 'name': 'DigitalKisanBot', 'tweets': 1453}]

In [76]:
# Tercera función
def days_with_more_tweets(tweets):
    def more_tweets(day):
        return day["tweets"]
    
    days = {}
    with_more_tweets = []
    
    for tweet in tweets:
        tweet_date = tweet["date"][:10]
        if tweet_date in days:
            days[tweet_date] += 1
        else:
            days[tweet_date] = 1
                
    for _day in days:
        day = {"date": _day, "tweets": days[_day]}
        if len(with_more_tweets) <= 9:
              with_more_tweets.append(day)
              with_more_tweets = sorted(with_more_tweets, key=more_tweets)
        else: 
            if with_more_tweets[9]["tweets"] <= day["tweets"]:
                with_more_tweets.pop(0)
                with_more_tweets.append(day)
            elif with_more_tweets[0]["tweets"] <= day["tweets"]:
                with_more_tweets.append(day)
                with_more_tweets = sorted(with_more_tweets, key=more_tweets)[1:]

    
    return with_more_tweets

In [86]:
days_with_more_tweets(tweets)

[{'date': '2021-02-03', 'tweets': 83866},
 {'date': '2021-02-04', 'tweets': 58607},
 {'date': '2021-02-05', 'tweets': 33332},
 {'date': '2021-02-02', 'tweets': 28548},
 {'date': '2021-02-06', 'tweets': 22420},
 {'date': '2021-02-07', 'tweets': 11325},
 {'date': '2021-02-09', 'tweets': 9320},
 {'date': '2021-02-08', 'tweets': 8920},
 {'date': '2021-02-10', 'tweets': 7973},
 {'date': '2021-02-11', 'tweets': 5698}]

In [107]:
# Cuarta función
def hashtags_with_more_tweets(tweets):
    
    
    hashtags = {}
    
    for tweet in tweets:
        for hashtag in set(re.findall("#[a-zA-Z]+", tweet["content"])):
            if hashtag in hashtags:
                hashtags[hashtag] += 1
            else:
                hashtags[hashtag] = 1
    
    return sorted([{"hashtag": h, "tweets": hashtags[h]} for h in hashtags], key=lambda i: i["tweets"])[-1:-10:-1]

In [108]:
hashtags_with_more_tweets(tweets)

[{'hashtag': '#FarmersProtest', 'tweets': 400410},
 {'hashtag': '#IStandWithFarmers', 'tweets': 15745},
 {'hashtag': '#farmersprotest', 'tweets': 15407},
 {'hashtag': '#IndianFarmersHumanRights', 'tweets': 11859},
 {'hashtag': '#FarmersAreIndia', 'tweets': 10947},
 {'hashtag': '#StandWithFarmers', 'tweets': 10577},
 {'hashtag': '#Rihanna', 'tweets': 9024},
 {'hashtag': '#FarmersProtests', 'tweets': 8763},
 {'hashtag': '#Farmers', 'tweets': 8100}]

In [ ]:
# Main
def main():
    pass